## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50

In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 以訓練好的 ResNet50 為基礎來建立模型
model = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(32,32,3)) 
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10# 訓練整個資料集共 30個循環
FREEZE_LAYERS = 2 # 凍結網路層數

from keras.callbacks import ModelCheckpoint

model_ckpt = ModelCheckpoint(filepath="./tmp.h5", 
                             monitor="val_loss", 
                             save_best_only=True)

train_augment_generator = ImageDataGenerator(
       rotation_range=10,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True)

test_augment_generator = ImageDataGenerator(
       rotation_range=10,
       width_shift_range=0.1,
       height_shift_range=0.1,
       horizontal_flip=True)

x = model.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
model_final = Model(inputs=model.input, outputs=output_layer)
for layer in model_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model_final.fit_generator(train_augment_generator.flow(x_train, y_train, batch_size=batch_size),
                           steps_per_epoch=int(len(x_train)/batch_size),
                           epochs=epochs,
                           verbose=1,
                           validation_data=test_augment_generator.flow(x_test, y_test, batch_size=batch_size),
                           validation_steps=int(len(x_test)/batch_size),
                           callbacks=[model_ckpt])
score = model_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
390/390 [==============================] - 3768s 10s/step - loss: 1.7072 - accuracy: 0.4898 - val_loss: 5.5693 - val_accuracy: 0.1002
Epoch 2/10
390/390 [==============================] - 3742s 10s/step - loss: 1.3941 - accuracy: 0.5742 - val_loss: 3.6898 - val_accuracy: 0.1698
Epoch 3/10
390/390 [==============================] - 3767s 10s/step - loss: 1.3452 - accuracy: 0.5803 - val_loss: 1.7578 - val_accuracy: 0.3769
Epoch 4/10
390/390 [==============================] - 3767s 10s/step - loss: 1.2698 - accuracy: 0.5971 - val_loss: 7.3050 - val_accuracy: 0.5222
Epoch 5/10
390/390 [==============================] - 3809s 10s/step - loss: 1.2164 - accuracy: 0.6363 - val_loss: 1.1584 - val_accuracy: 0.6079
Epoch 6/10
390/390 [==============================] - 3807s 10s/step - loss: 1.0866 - accuracy: 0.6540 - val_loss: 1.1871 - val_accuracy: 0.6136
Epoch 7/10
390/390 [==============================] - 